# Download Meteorological Station Data and Interpolate for Sample Sites

## Install and import necessary packages

In [1]:
# Install packages
# ! pip install cdo-api-py
# ! pip install geopy
# !pip install openpyxl

# import packages
from cdo_api_py import Client
import pandas as pd
from datetime import datetime
from pprint import pprint
import os
import geopy.distance
import requests
import json
import numpy as np

In [6]:
# Print current directory
print('Current Directory: ' + os.getcwd())

# Define location of bioscan data
bioscan_file = os.path.dirname(os.getcwd()) + '/Data_Upon_Request/bioscan.xlsx'
print('Bioscan Data Directory: ' + bioscan_file)

Current Directory: /Users/teaganbaiotto/Mirror/Research/Lewthwaite_et_al_2023/Step01_Data_Preperation
Bioscan Data Directory: /Users/teaganbaiotto/Mirror/Research/Lewthwaite_et_al_2023/Data_Upon_Request/bioscan.xlsx


## Read sampling location data from bioscan sheet

In [7]:
# Get dataframe of sites and coordinates from bioscan data sheet
bioscan_locations = pd.read_excel(open(bioscan_file, 'rb'), sheet_name='Annual Combined All')[["Site", "Latitude", "Longitude"]]
bioscan_locations.index = bioscan_locations.iloc[:,0]
bioscan_locations = bioscan_locations.drop(columns="Site")
bioscan_locations = bioscan_locations[~bioscan_locations.index.duplicated(keep='first')]
bioscan_locations

,Latitude,Longitude
Site,,
Site01,34.01707,-118.28868
Site03,34.03400,-118.28100
Site04,34.11800,-118.28400
Site05,34.09300,-118.27400
Site06,34.11600,-118.27900
...,...,...
Site59,33.93200,-118.25900
Site60,34.24700,-118.50800
Site61,34.24400,-118.57900


## Get list of all met stations in LA

List of meteorological stations and data are downloaded using the Climate Data Online API available through NOAA: https://www.ncei.noaa.gov/cdo-web/datasets

In [8]:
# Get your access token from here: https://www.ncdc.noaa.gov/cdo-web/token

token = ""     # be sure not to share your token publicly
my_client = Client(token, default_units=None, default_limit=1000)

In [9]:
# Information for CDO API for gathering which meteorological stations are within boundary area and what data you wish to access

# Extent of bounding area - Default is boundary of LA county
extent = {
    "north": 34.823302,
    "south": 32.75004,
    "east": -117.646374,
    "west": -118.951721,
}

# Start and end dates for daily meteorological data
start_date = "2014-01-01"
end_date = "2018-12-31"

# Dataset you are interested in accessing
datasetid='GHCND'

# Daily: GHCND

# What data variable are you interested in accessing
datatypeid='PRCP'

# Average wind speed: AWND
# Fastest 5 second wind speed: WSF5
# Precipitation: PRCP
# Average temp: TAVG
# Min temp: TMIN
# Max temp: TMAX
# All available daily variables: https://www.ncei.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf

# Input IDW parameters - number of sites to include in calculation and power
num_sites_IDW = 5
IDW_power = 1

# View all datatypes
# pprint(my_client.list_datatypes())

In [10]:
# Get list of met stations in LA county
stations = my_client.find_stations(
    datasetid=datasetid,
    extent=extent,
    startdate=datetime.strptime(start_date, '%Y-%m-%d'),
    enddate=datetime.strptime(end_date, '%Y-%m-%d'),
    datatypeid=datatypeid,
    return_dataframe=True)
stations

https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&startdate=2014-01-01&enddate=2018-12-31&extent=32.75004&extent=-118.951721&extent=34.823302&extent=-117.646374&datatypeid=PRCP&limit=1000


,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,155.8,2008-09-23,2023-06-27,34.168900,"GLENDALE 2.4 WSW, CA US",1.0000,GHCND:US1CALA0001,METERS,-118.294700
1,88.1,2008-10-16,2019-12-04,33.937285,"WHITTIER 3.6 ESE, CA US",0.1124,GHCND:US1CALA0002,METERS,-117.984379
2,61.0,2008-10-31,2022-06-04,33.790180,"TORRANCE 2.8 SW, CA US",0.9629,GHCND:US1CALA0003,METERS,-118.336890
3,403.3,2008-11-10,2023-06-12,34.187426,"ALTADENA 0.7 ESE, CA US",0.6062,GHCND:US1CALA0005,METERS,-118.124477
4,207.9,2008-12-11,2017-03-06,34.078600,"MONTE NIDO 0.2 SSW, CA US",0.8517,GHCND:US1CALA0008,METERS,-118.687000
...,...,...,...,...,...,...,...,...,...
124,1374.6,1934-07-01,2023-07-23,34.743610,"SANDBERG, CA US",0.9870,GHCND:USW00023187,METERS,-118.725280
125,477.0,1948-01-01,2023-07-23,33.404190,"AVALON CATALINA AIRPORT, CA US",0.4443,GHCND:USW00023191,METERS,-118.414560
126,54.6,1906-04-01,2023-07-23,34.023600,"LOS ANGELES DOWNTOWN USC, CA US",0.9325,GHCND:USW00093134,METERS,-118.291100
127,13.0,1999-02-12,2023-07-23,33.679750,"SANTA ANA JOHN WAYNE AIRPORT, CA US",0.9984,GHCND:USW00093184,METERS,-117.867460


In [17]:
# Create dataframe of station names and their lat/long locations only
stations_locations = stations[["id", "latitude", "longitude"]].rename(columns={"latitude": "Latitude", "longitude": "Longitude"})
stations_locations.index = stations_locations.iloc[:,0]
stations_locations = stations_locations.drop(columns="id")
stations_locations

,Latitude,Longitude
id,,
GHCND:US1CALA0001,34.168900,-118.294700
GHCND:US1CALA0002,33.937285,-117.984379
GHCND:US1CALA0003,33.790180,-118.336890
GHCND:US1CALA0005,34.187426,-118.124477
GHCND:US1CALA0008,34.078600,-118.687000
...,...,...
GHCND:USW00023187,34.743610,-118.725280
GHCND:USW00023191,33.404190,-118.414560
GHCND:USW00093134,34.023600,-118.291100


## Compute distance matrix of each sampling location to each met station

In [31]:
# Create dictionaries from bioscan site dataframe and met station dataframe for easy computing
bioscan_dict = bioscan_locations.to_dict(orient="index")
stations_dict = stations_locations.to_dict(orient="index")

# for each site - met station combo, compute distance
for sample,bioscan_coord in bioscan_dict.items():
    # coordinates of bioscan site
    sample_coords = (bioscan_coord["Latitude"], bioscan_coord["Longitude"])
    
    for station,station_coord in stations_dict.items():
        # coordinates of met station
        station_coords = (station_coord["Latitude"], station_coord["Longitude"])
        # compute distance between site and met station coordinates in kilometers
        dist_to_station = geopy.distance.distance(sample_coords, station_coords).km
        # assign distance to dictionary
        bioscan_dict[sample][station] = dist_to_station

#convert distance dictionary to pandas df
dist_to_stations = pd.DataFrame(bioscan_dict).T.iloc[: , 2:]
dist_to_stations

# FOR PRCP:
dist_to_stations = dist_to_stations.drop('GHCND:US1CALA0008', axis=1)
dist_to_stations = dist_to_stations.drop('GHCND:USC00041194', axis=1)
# dist_to_stations = dist_to_stations.drop('GHCND:US1CAVT0014', axis=1)
# dist_to_stations = dist_to_stations.drop('GHCND:US1CAOR0022', axis=1)


In [32]:
# Rank the distances, assign 1 to lowerst value, n to highest
ranks = dist_to_stations.rank(axis=1, method='first')
ranks

,GHCND:US1CALA0001,GHCND:US1CALA0002,GHCND:US1CALA0003,GHCND:US1CALA0005,GHCND:US1CALA0009,GHCND:US1CALA0010,GHCND:US1CALA0011,GHCND:US1CALA0014,GHCND:US1CALA0015,GHCND:US1CALA0021,...,GHCND:USW00023129,GHCND:USW00023130,GHCND:USW00023152,GHCND:USW00023174,GHCND:USW00023182,GHCND:USW00023187,GHCND:USW00023191,GHCND:USW00093134,GHCND:USW00093184,GHCND:USW00093197
Site01,11.0,44.0,35.0,28.0,15.0,18.0,27.0,69.0,86.0,43.0,...,38.0,41.0,20.0,7.0,112.0,127.0,108.0,1.0,84.0,8.0
Site03,9.0,45.0,37.0,23.0,14.0,19.0,32.0,66.0,84.0,36.0,...,39.0,40.0,18.0,8.0,106.0,127.0,112.0,1.0,85.0,10.0
Site04,2.0,44.0,51.0,15.0,7.0,33.0,45.0,52.0,76.0,31.0,...,48.0,27.0,6.0,29.0,97.0,120.0,118.0,4.0,98.0,22.0
Site05,4.0,44.0,49.0,19.0,8.0,28.0,43.0,60.0,77.0,30.0,...,47.0,31.0,7.0,25.0,99.0,124.0,118.0,3.0,96.0,20.0
Site06,3.0,44.0,50.0,15.0,8.0,32.0,45.0,54.0,76.0,31.0,...,48.0,29.0,6.0,28.0,97.0,120.0,119.0,4.0,98.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Site59,34.0,31.0,15.0,42.0,38.0,16.0,11.0,87.0,84.0,45.0,...,13.0,52.0,44.0,3.0,120.0,127.0,95.0,2.0,67.0,19.0
Site60,19.0,72.0,61.0,39.0,5.0,57.0,56.0,17.0,91.0,52.0,...,70.0,2.0,8.0,41.0,68.0,69.0,114.0,34.0,106.0,25.0
Site61,25.0,72.0,60.0,46.0,9.0,61.0,54.0,22.0,91.0,56.0,...,69.0,3.0,15.0,39.0,70.0,64.0,108.0,36.0,106.0,26.0
Site62,2.0,58.0,55.0,27.0,3.0,41.0,49.0,38.0,85.0,39.0,...,60.0,10.0,1.0,32.0,82.0,107.0,117.0,14.0,105.0,15.0


## Download meteorological data from stations

In [27]:
# SUGGESTED METHOD PER API INSTRUCTIONS, DOES NOT WORK - UNABLE TO DOWNLOAD DATA FOR MONTHS OF JAN AND FEB
# station_data_dict = {}
# for rowid, station in stations.iterrows():  # remember this is a pandas dataframe!
#     station_data = my_client.get_data_by_station(
#         datasetid=datasetid,
#         stationid=station['id'],
#         startdate=startdate,
#         enddate=enddate,
#         return_dataframe=True,
#         include_station_meta=True   # flatten station metadata with ghcnd readings
#     )
#     pprint(station_data)
    
#     station_data_dict[station['id']] = station_data
    
# Download meteorological data for all stations in bbox within time range of interest and put in dictionary of dataframes
station_data_dict = {}
for rowid, station in stations.iterrows():
    # Request data from NOAA NCO
    resp = requests.get(f"https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes={datatypeid}&stations={station['id'].split(':')[-1]}&startDate={start_date}&endDate={end_date}&units=standard", headers={"Token": token})
    resp_contents = resp.content.decode("utf-8").splitlines()
    resp_contents = [line.replace('"', '') for line in resp_contents]
    
    # Format data in pandas df
    resp_df = pd.DataFrame([sub.split(",") for sub in resp_contents])
    resp_df.columns=resp_df.iloc[0]
    resp_df = resp_df.drop(resp_df.index[0]).replace(r'^\s*$', np.nan, regex=True)
    
    # Assign df to station id in dictionary
    station_data_dict[station['id']] = resp_df

In [37]:
# Change index of each dataframe from integers to date

# For PRCP issue, run following lines too 
station_data_dict.pop('GHCND:US1CALA0008')
station_data_dict.pop('GHCND:USC00041194')


for station_name,data in station_data_dict.items():
    try:
        data.index = pd.to_datetime(data.iloc[: , 1])
    except:
        print('Must remove: ', station_name)

# station_data_dict

KeyError: 'GHCND:US1CALA0008'

In [ ]:
# save as file as backup
with open('stations.txt','w') as data: 
      data.write(str(station_data_dict))

## Interpolate using inverse-distance weighting (IDW) for each sample site for all dates

Read more about IDW here: https://www.geodose.com/2019/03/spatial-interpolation-inverse-distance-weighting-idw.html

In [ ]:
# Create range of all dates in bioscan data period
date_range = pd.date_range(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d')).tolist()

# create empty df to store outputs of IDW
IDW_output = pd.DataFrame(columns=ranks.index.values.tolist(), index=date_range)
    
for site in ranks.index.values.tolist():
    for date in date_range:
        station_rank = 1.0
        num_selected = 0

        # create empty df for distance (weights) and met variable values to be put for each of x number of sites used for IDW
        date_values = pd.DataFrame(columns=["distance", "value"])
        while num_selected < num_sites_IDW:
            # get site name with rank equal to "station_rank" - starts at 1 and increases by 1 each iteration
            my_station = ranks.loc[[site]].eq(station_rank, axis=0).idxmax(axis=1)
            # skip this met station if date is not in dataset index
            if date in station_data_dict[my_station[0]].index:
                # skip this met station if value is nan for date
                if bool(station_data_dict[my_station[0]].loc[[date], [datatypeid]].notna()[datatypeid][0]):
                    date_values.loc[len(date_values.index)] = [dist_to_stations.loc[[site], [my_station[0]]][my_station[0]][0],station_data_dict[my_station[0]].loc[[date], [datatypeid]][datatypeid][0]] 
                    # since this met station had data for this date, it counts towards one of the x number of stations that will be used to compute IDW for value
                    num_selected += 1
                else:
                    pass
            else:
                pass

            station_rank += 1

        # IDW Calculation for site, date combo
        date_values['weight'] = 1/(date_values['distance']**(IDW_power))
        date_values['IDW_top'] = date_values['value'].astype(float) * date_values['weight']
        IDW_value = date_values.sum()['IDW_top']/date_values.sum()['weight']

        # store IDW value for site, date into IDW_output dataframe
        IDW_output.loc[date,site] = IDW_value
    print(f'Done with {site}')

Done with Site01
Done with Site03
Done with Site04
Done with Site05
Done with Site06
Done with Site07
Done with Site08
Done with Site09
Done with Site10
Done with Site11
Done with Site12
Done with Site13
Done with Site14
Done with Site15
Done with Site16
Done with Site17
Done with Site18
Done with Site19
Done with Site20
Done with Site21
Done with Site22
Done with Site23
Done with Site24
Done with Site25
Done with Site26
Done with Site27
Done with Site28
Done with Site29
Done with Site30
Done with Site31
Done with Site32
Done with Site33
Done with Site34
Done with Site35
Done with Site36
Done with Site37
Done with Site38
Done with Site39
Done with Site40
Done with Site41
Done with Site42
Done with Site43
Done with Site44
Done with Site45
Done with Site46
Done with Site47
Done with Site48
Done with Site49
Done with Site50
Done with Site51
Done with Site52
Done with Site53
Done with Site54
Done with Site55
Done with Site56
Done with Site57
Done with Site58
Done with Site59
Done with Site

In [ ]:
# Convert columns to float and round
IDW_output = IDW_output.astype('float').round(2)

# Save site IDW calculation as csv
IDW_output.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'met_station_data', f'{datatypeid}.csv'), index=True)

IDW_output